# Exploratory Data Analysis of Vitamin D-Related Transcriptional Signatures

This notebook presents the exploratory data analysis (EDA) of transcriptional responses to vitamin D and its analogs, based on data from the LINCS L1000 dataset. The dataset includes gene expression signatures from human cell lines exposed to different vitamin D-related compounds across various doses and time points.

The goal of this analysis is to characterize the diversity and distribution of the selected signatures, examine the experimental conditions (cell lines, doses, exposure times), and identify preliminary patterns in gene expression profiles that may inform subsequent modeling and biological interpretation.

In [1]:
# Import libraries for data manipulation, visualization, and dimensionality reduction
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Configuración de visualización
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 6)

In [3]:
# Paths to pre-filtered files
EXPR_PATH = "../processed_data/vitD_expression_matrix.csv"
META_PATH = "../processed_data/siginfo_vitD_filtered.csv"

# Load expression matrix (genes x signatures)
exp_df = pd.read_csv(EXPR_PATH, index_col=0)

# Load metadata associated with the signatures
meta_df = pd.read_csv(META_PATH, index_col="sig_id")

# Display shapes
print("Expression matrix shape:", exp_df.shape)
print("Metadata shape:", meta_df.shape)


Expression matrix shape: (12328, 258)
Metadata shape: (258, 37)


In [4]:
# Confirm all signature IDs in metadata are present in the expression matrix
assert set(meta_df.index).issubset(set(exp_df.columns)), "Some sig_id in metadata are missing from expression matrix"

# Reorder expression matrix columns to match metadata index
exp_df = exp_df[meta_df.index]

# Final confirmation
print("Column order aligned:", all(exp_df.columns == meta_df.index))

Column order aligned: True


In [7]:
# Display a subset of the expression matrix to inspect gene-signature structure
display(exp_df.iloc[:5, :5])

# Display the first rows of the metadata to examine available annotations
display(meta_df.head())

# Calculate the number of unique compounds present in the filtered dataset
n_compounds = meta_df["cmap_name"].nunique()

# Summarize the number of transcriptional signatures available per compound
compound_counts = meta_df["cmap_name"].value_counts()
display(compound_counts)

,CPD002_MCF7_24H:BRD-K96390176-001-01-1:10,CPD002_PC3_24H:BRD-K96390176-001-01-1:10,ASG002_MCF7_24H:O08,ASG003_MCF7_24H:G20,ASG003_U2OS_24H:G20
rid,,,,,
10,-0.442904,0.089615,-0.092857,-0.348907,0.476497
100,1.073123,0.568674,-0.657938,-0.666472,-0.588692
1000,0.231224,-0.459265,-0.677316,0.674801,-0.248705
10000,-0.857503,0.666649,-0.185645,0.493368,0.835636
10001,0.514508,-0.984091,-0.306706,-0.101548,-0.573884


,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_itime,pert_time,pert_time_unit,cell_mfc_name,pert_mfc_id,...,det_wells,det_plates,distil_ids,build_name,project_code,cmap_name,is_exemplar_sig,is_ncs_sig,is_null_sig,merge_key
sig_id,,,,,,,,,,,,,,,,,,,,,
CPD002_MCF7_24H:BRD-K96390176-001-01-1:10,b6,10.00,10.00000,uM,10 uM,24 h,24.0,h,MCF7,BRD-K96390176,...,K07,CPD002_MCF7_24H_X1_B6_DUO52HI53LO|CPD002_MCF7_...,CPD002_MCF7_24H_X1_B6_DUO52HI53LO:K07|CPD002_M...,NaN,CPD,calcipotriol,1,1.0,0.0,BRD-K96390176_MCF7_24.0
CPD002_PC3_24H:BRD-K96390176-001-01-1:10,b6,10.00,10.00000,uM,10 uM,24 h,24.0,h,PC3,BRD-K96390176,...,K07,CPD002_PC3_24H_X1_B6_DUO52HI53LO|CPD002_PC3_24...,CPD002_PC3_24H_X1_B6_DUO52HI53LO:K07|CPD002_PC...,NaN,CPD,calcipotriol,1,1.0,0.0,BRD-K96390176_PC3_24.0
ASG002_MCF7_24H:O08,b34,1.11,1.00000,uM,1.11 uM,24 h,24.0,h,MCF7,BRD-K27316855,...,O08,ASG002_MCF7_24H_X1_B34|ASG002_MCF7_24H_X3_B34|...,ASG002_MCF7_24H_X1_B34:O08|ASG002_MCF7_24H_X3_...,NaN,ASG,calcitriol,0,1.0,0.0,BRD-K27316855_MCF7_24.0
ASG003_MCF7_24H:G20,b41,1.11,1.00001,uM,1.11 uM,24 h,24.0,h,MCF7,BRD-K56429665,...,G20,ASG003_MCF7_24H_X1_B41|ASG003_MCF7_24H_X2.A2_B...,ASG003_MCF7_24H_X1_B41:G20|ASG003_MCF7_24H_X2....,NaN,ASG,calcipotriol,0,1.0,0.0,BRD-K56429665_MCF7_24.0
ASG003_U2OS_24H:G20,b41,1.11,1.00001,uM,1.11 uM,24 h,24.0,h,U2OS,BRD-K56429665,...,G20,ASG003_U2OS_24H_X1_B41|ASG003_U2OS_24H_X2.A2_B...,ASG003_U2OS_24H_X1_B41:G20|ASG003_U2OS_24H_X2....,NaN,ASG,calcipotriol,0,1.0,0.0,BRD-K56429665_U2OS_24.0


cmap_name
calcitriol      91
maxacalcitol    34
ercalcitriol    32
calcipotriol    30
tacalcitol      28
seocalcitol     27
paricalcitol    16
Name: count, dtype: int64